Troisième base de données : challenge Paris-Saclay

16/12/24 : je vais continuer à travailler sur le dataset no 2, nous verrons plus tard comment utiliser le dernier.

Importation des bibliothèques et du dataset

In [15]:
import pandas as pd 
import numpy as np
import folium

In [11]:
df_real_time=pd.read_csv("ytrain_NpxebDC.csv")
df_real_time.head(20)
df_real_time.dropna()

df_static=pd.read_csv("info_static.csv")
df_static

/tmp/ipykernel_98107/2704645367.py:1: DtypeWarning: Columns (1,3,5,11,20,49,52,54,55,56,64,70,71,94,95,101,102,114,129,131,132,133,138,139,152,157,163,174,180,184,193,196,201,207,208,219,220,228,241,245,250,256,258,264,265,269,270,271,272,273) have mixed types. Specify dtype option on import or set low_memory=False.
  df_real_time=pd.read_csv("ytrain_NpxebDC.csv")


,t_id,s_id,latitude,longitude,street,city,zipcode,model
0,S84-T1,S84,48.863766,2.411770,104 RUE LOUIS LUMIERE,Paris,75020,"E/F, Combo, T2, CHAdeMO, T3"
1,S84-T3,S84,48.863766,2.411770,104 RUE LOUIS LUMIERE,Paris,75020,"CHAdeMO, T2, T3, E/F, Combo"
2,S86-T1,S86,48.890610,2.354230,27 RUE ORDENER,Paris,75018,"CHAdeMO, Combo, T2, T3, E/F"
3,S86-T2,S86,48.890610,2.354230,27 RUE ORDENER,Paris,75018,E/F
4,S86-T3,S86,48.890610,2.354230,27 RUE ORDENER,Paris,75018,"T3, Combo, CHAdeMO, E/F, T2"
...,...,...,...,...,...,...,...,...
268,S41-T2,S41,48.869370,2.303269,55 RUE PIERRE CHARRON,Paris,75008,E/F
269,S7-T2,S7,48.853445,2.386880,206 Boulevard Voltaire,Paris,75011,E/F
270,S57-T2,S57,48.885689,2.297463,4 AV GOURGAUD,Paris,75017,E/F
271,S47-T2,S47,48.820007,2.365661,107 BOULEVARD MASSENA,Paris,75013,E/F


In [14]:
distinct_elmts=df_real_time.apply(lambda col : col.unique())
df_real_time.replace("nan",np.nan,inplace=True)
count_nan=df_real_time.isnull().sum()

stations_list=df_static.apply(lambda col : col.unique())
stations_list["street"]

array(['104 RUE LOUIS LUMIERE', '27 RUE ORDENER',
       '150 RUE DU FAUBOURG SAINT MARTIN', '5 RUE DE REAUMUR',
       '4 Rue Changarnier', '9 PLACE DU COLONEL FABIEN', '47 RUE DIDOT',
       '55 RUE MONGE', '4 PLACE ARMAND CARREL', '5 Rue Docteur Goujon',
       '103 RUE CAULAINCOURT', '25 RUE DE VOUILLE',
       '36 AVENUE DES TERNES', '35 RUE DU BAC', '150 RUE DES POISSONIERS',
       '81 RUE DE ROME', '54 Rue Baron Leroy', '4 QUAI DU MARCHE NEUF',
       '66 RUE DE RENNES', '4 AVENUE MARCEL DORET',
       '137 BD VINCENT AURIOL', '7 BOULEVARD DES CAPUCINES',
       '3 AVENUE DU GENERAL DODDS', '10 Avenue de Flandre',
       '24 AVENUE PAUL DOUMER', '2 Rue Antonin Mercié',
       '14 rue de Crimée', '77 AVENUE DE LA GRANDE ARMEE',
       '45 AVENUE VICTOR HUGO', '9 AVENUE DE VILLARS', '22 RUE FABERT',
       '55 RUE DESNOUETTES', '21 Rue Berbier du Mets', '3 AV DUTUIT',
       '126 BOULEVARD DIDEROT', '46 AVENUE REILLE', '2 Rue Abel',
       '2 RUE PERIGNON', '44 RUE ETIENNE MARCEL

Importing Kmeans, KNN functions

In [4]:
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split



Indicator for average availability

In [5]:
R= 6371.0 #radius of the Earth in km
def haversine(lat1,lon1,lat2,lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance




In [7]:
radius=1 #check for availability in a 1 km radius
lat_stations=df_static["latitude"].unique()
lon_stations=df_static["longitude"].unique()
coord_stations=np.column_stack((lat_stations,lon_stations))

coord_stations

array([[48.863766 ,  2.41177  ],
       [48.89061  ,  2.35423  ],
       [48.875465 ,  2.360103 ],
       [48.864719 ,  2.359043 ],
       [48.8452653,  2.4112383],
       [48.878799 ,  2.368851 ],
       [48.831448 ,  2.320052 ],
       [48.84477  ,  2.35242  ],
       [48.882794 ,  2.382705 ],
       [48.839947 ,  2.397822 ],
       [48.889731 ,  2.33891  ],
       [48.834705 ,  2.305529 ],
       [48.878965 ,  2.293606 ],
       [48.857124 ,  2.327753 ],
       [48.897671 ,  2.352758 ],
       [48.882718 ,  2.319768 ],
       [48.833603 ,  2.389173 ],
       [48.854057 ,  2.34556  ],
       [48.8521255,  2.3313459],
       [48.837055 ,  2.259932 ],
       [48.832893 ,  2.363025 ],
       [48.870628 ,  2.33321  ],
       [48.834084 ,  2.405248 ],
       [48.8847205,  2.3699612],
       [48.861016 ,  2.28308  ],
       [48.828454 ,  2.3006428],
       [48.8778   ,  2.39182  ],
       [48.876162 ,  2.286403 ],
       [48.871465 ,  2.289894 ],
       [48.852665 ,  2.31368  ],
       [48

In [ ]:
m=folium.Map(location=(48.863766 ,  2.41177  ))
for i in range(len(coord_stations)):
    folium.Marker(location=coord_stations[i]).add_to(m)
    
def availability_status(station,t):
    
    

,timestamp,S7-T1,S2-T1,S19-T1,S56-T3,S85-T3,S16-T3,S16-T1,S94-T3,S28-T1,...,S47-T2,S65-T2,S32-T2,S21-T2,S13-T2,S97-T2,S25-T1,S25-T2,S98-T2,S99-T2
0,2019-11-25T00:00:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,...,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
1,2019-11-25T00:15:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,...,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
2,2019-11-25T00:30:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,...,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
3,2019-11-25T00:45:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,...,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
4,2019-11-25T01:00:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,...,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31227,2020-11-08T22:45:00+00:00,NaN,Available,NaN,Available,NaN,Available,Available,Available,Available,...,NaN,NaN,Available,Available,Available,Available,Available,Available,Available,Available
31228,2020-11-08T23:00:00+00:00,NaN,Available,NaN,Available,NaN,Available,Available,Available,Available,...,NaN,NaN,Available,Available,Available,Available,Available,Available,Available,Available
31229,2020-11-08T23:15:00+00:00,NaN,Available,NaN,Available,NaN,Available,Available,Available,Available,...,NaN,NaN,Available,Available,Available,Available,Available,Available,Available,Available
31230,2020-11-08T23:30:00+00:00,NaN,Available,NaN,Available,NaN,Available,Available,Available,Available,...,NaN,NaN,Available,Available,Available,Available,Available,Available,Available,Available


In [ ]:
def knn_status(lat,lon,date):
    status=[]
    for s in coord_stations:
        if haversine(lat,lon,s[0],s[1])<= radius:
            status.append(s)